In [49]:
import pandas as pd
from boto3 import Session
import boto3

session = Session()
credentials = session.get_credentials()

current_credentials = credentials.get_frozen_credentials()

client = boto3.client(
    's3',
    aws_access_key_id = current_credentials.access_key,
    aws_secret_access_key = current_credentials.secret_key,
    aws_session_token=current_credentials.token)

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [4]:
# Get training data that needs to move
trainImgKeys = pd.read_csv('collage_gan_training_data.csv',index_col=0)

In [3]:
source_bucket = 'tellisa-collage-gan'
destination_bucket = 'tellisa-collage-gan'

In [13]:
source_key=trainImgKeys.iloc[0]['Key']

In [26]:
#Testing the copy object method
client.copy_object(Bucket=destination_bucket,CopySource=source_bucket+'/'+source_key,Key='collage_gan_subset'+source_key[11:])

{'ResponseMetadata': {'RequestId': 'TX6FAJCWXWNPVWP7',
  'HostId': 'VWnntCPdXutDCEE47BSSpBCuP9jrJmhoMmwPSQUmJ+9QVAcBnvwESfDPGFFkGiO0GpSJeaDqAvg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'VWnntCPdXutDCEE47BSSpBCuP9jrJmhoMmwPSQUmJ+9QVAcBnvwESfDPGFFkGiO0GpSJeaDqAvg=',
   'x-amz-request-id': 'TX6FAJCWXWNPVWP7',
   'date': 'Sun, 22 Jan 2023 23:13:33 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'content-type': 'application/xml',
   'server': 'AmazonS3',
   'content-length': '234'},
  'RetryAttempts': 0},
 'ServerSideEncryption': 'AES256',
 'CopyObjectResult': {'ETag': '"77fb71617b0a87bb0135ee115e421335"',
  'LastModified': datetime.datetime(2023, 1, 22, 23, 13, 33, tzinfo=tzlocal())}}

In [27]:
#Move files to new location for training
for i in range(len(trainImgKeys)):
    source_key=trainImgKeys.iloc[i]['Key']
    client.copy_object(Bucket=destination_bucket,CopySource=source_bucket+'/'+source_key,Key='collage_gan_subset'+source_key[11:])

In [2]:
#Convert Data in Structured CSV
def get_image(img_key):
    bucket_name = 'tellisa-collage-gan'
    img_inst = client.get_object(Bucket=bucket_name,Key=img_key)
    return img_inst['Body'].read()

In [4]:
# Reused code from Image Embed script to get keys
bucket_name = 'tellisa-collage-gan'
prefix = 'collage_gan_subset/train/'

paginator = client.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name,Prefix=prefix) 

objKeyFrame = pd.DataFrame()
for page in pages:
    objKeyFrame = pd.concat([objKeyFrame,pd.DataFrame(page['Contents'])],ignore_index=True )

In [6]:
from tensorflow.keras.preprocessing import image
objKeyFrame = objKeyFrame.iloc[1:]
objKeyFrame

,Key,LastModified,ETag,Size,StorageClass
1,collage_gan_subset/train/_.lunatico.__pic#0.jpg,2023-01-22 23:16:17+00:00,"""77fb71617b0a87bb0135ee115e421335""",50221,STANDARD
2,collage_gan_subset/train/_.lunatico.__pic#107.jpg,2023-01-22 23:16:19+00:00,"""af9172b5acd4b72366dd88b17bad1174""",41115,STANDARD
3,collage_gan_subset/train/_.lunatico.__pic#108.jpg,2023-01-22 23:22:19+00:00,"""c6b0eb4cd3806a8fb5b068682d8984fa""",76019,STANDARD
4,collage_gan_subset/train/_.lunatico.__pic#110.jpg,2023-01-22 23:17:49+00:00,"""7366eb5e717708677e8da67786bd2488""",88006,STANDARD
5,collage_gan_subset/train/_.lunatico.__pic#117.jpg,2023-01-22 23:16:58+00:00,"""7cc79aed90fd8c81d36cfd4ab69c4ea1""",38037,STANDARD
...,...,...,...,...,...
4996,collage_gan_subset/train/vertigo.artography_pi...,2023-01-22 23:23:44+00:00,"""67f18f8e96c09ff50418e0b748e22ef8""",25987,STANDARD
4997,collage_gan_subset/train/vertigo.artography_pi...,2023-01-22 23:18:20+00:00,"""75be32e7dbf97defa0525e1517b39de7""",54246,STANDARD
4998,collage_gan_subset/train/vertigo.artography_pi...,2023-01-22 23:18:10+00:00,"""e1a729ee36c6d7888683c1463aba5136""",47507,STANDARD
4999,collage_gan_subset/train/vertigo.artography_pi...,2023-01-22 23:18:08+00:00,"""2eb47a811d478ff480c83bc43294e681""",41046,STANDARD


In [11]:
# Handle Images
from PIL import Image
from io import BytesIO
import io
import cv2

#Handle multithreading
import concurrent.futures 

In [50]:
imageMetadata = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(get_image,objKeyFrame.Key)
    for result in results:
        imageMetadata.append(result)
        

def get_resized_image(img,img_dim):
    img = Image.open(io.BytesIO(img))
    arr = image.img_to_array(img)
    return pd.DataFrame(cv2.resize(arr,(img_dim,img_dim)).reshape(img_dim*img_dim*3)).T/255
    
imageFrame = pd.DataFrame()
for i in imageMetadata:
    imageFrame = pd.concat([imageFrame,get_resized_image(i,128)],ignore_index=True)



In [53]:
imageFrame.to_csv('train_data.csv')

In [55]:
from io import StringIO

csv_buffer = StringIO()
imageFrame.to_csv(csv_buffer)
client.put_object(Bucket=source_bucket, Key='collage_gan_subset/train_csv/train_data.csv', Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'A3CGWZ0YG25D9RTN',
  'HostId': 'C06JQnlWqIPBPYXyP72Kjka2edkFLU+AID0R9Ie+lP01i+ZQZ20G5la+oX63KUqXh5ywa0DdQS8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'C06JQnlWqIPBPYXyP72Kjka2edkFLU+AID0R9Ie+lP01i+ZQZ20G5la+oX63KUqXh5ywa0DdQS8=',
   'x-amz-request-id': 'A3CGWZ0YG25D9RTN',
   'date': 'Thu, 26 Jan 2023 00:59:59 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"401a5cb403d833ef8a2ca6409d68d2e6"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"401a5cb403d833ef8a2ca6409d68d2e6"',
 'ServerSideEncryption': 'AES256'}